## Hands-on session 1.3 - Fourier heat law - Solutions

Demonstrated concepts:

- Stationary variables
- Setting Dirichlet boundary conditions by excluding cells from operators
- Using the diffusion stencil for variables on the regular grid
- Calculation tree derivations
- Dashboard function for comparing multiple fluid variables

In [ ]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Basic setup

In [ ]:
#Wrapper initialization
rk = RKWrapper()

#I/O setuo
rk.jsonFilepath = "./config.json" 
hdf5Filepath = "./RMKOutput/day_1_3/"
rk.setHDF5Path(hdf5Filepath) 

#MPI setup
rk.setMPIData(numProcsX=4)

#Grid initialization
xGridWidths = 0.025*np.ones(512)
gridObj = Grid(xGridWidths, interpretXGridAsWidths=True)
rk.grid = gridObj

### The equations

We wish to solve the following well known equation

$$\frac{\partial T}{\partial t} = - \frac{\partial q}{\partial x}$$

$$q = -\kappa \frac{\partial T}{\partial x}$$

With some boundary conditions $T(x=x_0)=T_1$ and $T(x=x_N)=T_2$

Here we ignore any normalization issues and simply work in arbitrary units.

### Variables

We seem to have 2 equations, but only one time derivative. We could substitute $q$ into the equation for $T$ and get a second order equation, or we could have two first order equations. 

Let us add variables to test out both approaches.

In [ ]:
# Initial values, as well as the two boundary values for temperature
T = np.ones(512)
T[0] = 10 
T[-1] = 5

rk.addVarAndDual("T_diff",T,isCommunicated=True) # This will be our temperature for the diffusion implementation
rk.addVarAndDual("T_q",T,isCommunicated=True) # This will be our temperature for the separate q equation implementation

To add a variable that does not have a time derivative in its equation, we simply mark it as `stationary`

In [ ]:
rk.addVarAndDual("q",isStationary=True,primaryOnDualGrid=True,isCommunicated=True) # We still use a staggered grid

And finally the `time` variable

In [ ]:
rk.addVar("time",isScalar=True,isDerived=True)

## Calculation tree derivations

A particularly convenient way of producing derivation rules for ReMKiT1D, which is applicable in many situations, is to translate Python-like expressions into a tree represenation. 

In order to generate one such object, we first need to declare any variables we wish to use as Nodes. At the beginning of this notebook `Node` has been imported directly from ReMKiT1D. 

Let's suppose we have the following familiar expression for the conductivity $\kappa= \kappa_0 T^{3/2}$. This can be done as follows:

In [ ]:
nodeT = Node("T_diff_dual") #Note that we use T_diff_dual - this is because we need the conductivity on cell edges

kappa0 = 0.01
nodeKappa = kappa0 * nodeT**(3/2)

We can then manually add this derivation to the wrapper (we will need it for the diffusion stencil!)

In [ ]:
rk.addCustomDerivation("kappa_diff",treeDerivation(nodeKappa)) 

Let's also add a variable for $\kappa$, but using the other temperature 'T_q'. We can do it in an elegant 'one-liner', defining the corresponding derivation rule as the variable is added

In [ ]:
rk.addVar("kappa_q",
          isDerived=True,
          derivOptions=treeDerivation(kappa0*Node("T_q_dual")**(3/2)),
          derivationRule=sc.derivationRule("kappa_q",requiredVars=["T_q_dual"]),
          isCommunicated=True,
          isOnDualGrid=True)

The above will add the variable 'kappa_q' on the dual grid (where we need it), together with a derivation named "kappa_q" based on the `derivOptions` argument. 

The `requiredVars` argument to `derivationRule` is there to make sure that 'T_q_dual' is calculated before 'kappa_q'.

### On Dirichlet boundary conditions

ReMKiT1D by default does not use ghost cells to set Dirichlet boundary conditions. Instead, as the reader might have surmised, we have taken the first and the last cell in the domain and set them to some values. But how do we avoid evolving those cells?

The answer is in the `spatialProfile` argument to the `GeneralMatrixTerm` construct:

In [ ]:
help(sc.GeneralMatrixTerm)

The default profile, if the user does not supply one, is simply all ones. We will however, replace it by the following:

In [ ]:
spatialProfile = np.ones(512)
spatialProfile[0] = spatialProfile[-1] = 0
spatialProfile = spatialProfile.tolist() # Since the constructor requires a list! 

We will use this profile below for both models we're adding.

### The diffusion stencil approach

A prebuilt diffusion stencil is available using the following function

In [ ]:
sc.diffusionStencil?

As it can be seen, we need a defined derivation rule in order to use it. Fortunately, we have already added the custom derivation 'kappa_diff'.

The diffusion stencil corresponds to the operator $\nabla \kappa \nabla$, with the $\kappa$ associated with the derivation rule passed to the stencil.

Let us now build a model using this approach.

In [ ]:
newModel = sc.CustomModel("fourierDiff")

diffusionTerm = sc.GeneralMatrixTerm(evolvedVar="T_diff",
                                     implicitVar="T_diff",
                                     spatialProfile=spatialProfile,
                                     stencilData=sc.diffusionStencil("kappa_diff",reqVarNames=["T_diff_dual"],doNotInterpolate=True)) # We skip interpolating the diffusion coefficient because we already calculate it on the dual grid

newModel.addTerm("diffTerm",diffusionTerm)

rk.addModel(newModel)

### Stationary $q$ approach

Often the second order operator is not directly available, unlike with our diffusion example. In those situations we can either build a custom stencil, or resort to multiple first order equations. 

We begin by rewriting the $q$ equation so that all terms are on the RHS (further emphasizing the lack of a time derivative)

$$0 = -q -\kappa \nabla T$$

We see that we will need to add two terms for the q equation, including an identity term for $q$:

In [ ]:
newModel = sc.CustomModel("fourierQ")

identityTerm = sc.GeneralMatrixTerm(evolvedVar="q_dual", # When no implicit variable is supplied it defaults to the evolved variable
                                    customNormConst=-1,
                                    stencilData=sc.diagonalStencil()) # A diagonal stencil, effectively the Kronecker symbol

newModel.addTerm("q_identity",identityTerm)

The temperature gradient term is added using `staggeredGradStencil` and by setting the `varData` argument to use our added 'kappa_q' variable.

In [ ]:
gradTerm = sc.GeneralMatrixTerm(evolvedVar="q_dual",
                                implicitVar="T_q",# since this is a gradient, we want to use te cell centre values of T_q
                                customNormConst=-1,
                                varData=sc.VarData(reqRowVars=["kappa_q"]),
                                stencilData=sc.staggeredGradStencil())

newModel.addTerm("gradT",gradTerm)

Finally, we need the divergence of the heat flux in the temperature equation

In [ ]:
divQ = sc.GeneralMatrixTerm(evolvedVar="T_q",
                            implicitVar="q_dual",
                            customNormConst=-1,
                            spatialProfile=spatialProfile,          # Applying the same spatial profile as in the diffusion stencil approach
                            stencilData=sc.staggeredDivStencil()) 

newModel.addTerm("divQ",divQ)

rk.addModel(newModel)

### Time integration options

Following the approach from the previous sessions, we set the following:

In [ ]:
rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['T_q','T_diff','q_dual']) ) # we want all evolved variables to converge

rk.setIntegratorGlobalData(initialTimestep=0.1) 

bdeStep = sc.IntegrationStep("BE")

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(200)

### Write config file

In [ ]:
rk.writeConfigFile()

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Load data from ReMKiT1D output files

In [ ]:
numFiles = 50
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

### Explore data with built-in dashboard

In [ ]:
import panel as pn 
import RMK_support.dashboard_support as ds

pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show() # Removing .show() should display the dashboard inline - this can be buggy in some situations


We can also explore multiple fluid variables on a single graph using another dashboard function

In [ ]:
dashboard.fluidMultiComparison(["T_diff","T_q"])